# Import

In [1]:
import torch
import torchvision

from glcic.networks.completion_network import CompletionNetwork
from glcic.networks.discriminators import Discriminator
from glcic.trainers.conjugate_trainer import train
from glcic.utils import *

# Prepare

In [2]:
# load the dataset
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = get_dataloader(train_dataset_dir, train_dataset_dir+"barn/00001008.jpg", batch_size=48)

In [3]:
# others
cn = CompletionNetwork().cuda()
cn.load('../logs/models/cn_scrapped_weights')
cn_optimizer = torch.optim.Adadelta(cn.parameters())

discriminator = Discriminator().cuda()
discriminator_optimizer = torch.optim.Adadelta(discriminator.parameters())
_ = load_checkpoint("../logs/phase2_checkpoints/",discriminator, discriminator_optimizer)
discriminator_optimizer = torch.optim.Adadelta(discriminator.parameters()) # reset

num_batch = 10
replacement_val = torch.tensor([0.5062325495504219, 0.4255871700324652, 0.38299278586700136])

Load: load_state dict from ../logs/models/cn_scrapped_weights


# Test

In [4]:
# conjugate training
train(
    cn,
    discriminator,
    cn_optimizer,
    discriminator_optimizer,
    train_dataloader,
    num_batch,
    replacement_val,
    )


### BATCH 1/10 ###
loss: 1132.89111328125 for the completion loss, 6.556485459441319e-05 for the discriminator loss

### BATCH 2/10 ###
loss: 5045.57080078125 for the completion loss, 0.017460420727729797 for the discriminator loss

### BATCH 3/10 ###
loss: 2192.179931640625 for the completion loss, 0.0005946091841906309 for the discriminator loss

### BATCH 4/10 ###
loss: 2961.715576171875 for the completion loss, 0.003436347469687462 for the discriminator loss

### BATCH 5/10 ###
loss: 3596.602294921875 for the completion loss, 0.007216165773570538 for the discriminator loss

### BATCH 6/10 ###
loss: 2246.435546875 for the completion loss, 0.0027754181064665318 for the discriminator loss

### BATCH 7/10 ###
loss: 2410.891357421875 for the completion loss, 0.00043985299998894334 for the discriminator loss

### BATCH 8/10 ###
loss: 2315.976318359375 for the completion loss, 0.0004171373730059713 for the discriminator loss

### BATCH 9/10 ###
loss: 2318.715087890625 for the completion 

[(1132.89111328125, 6.556485459441319e-05),
 (5045.57080078125, 0.017460420727729797),
 (2192.179931640625, 0.0005946091841906309),
 (2961.715576171875, 0.003436347469687462),
 (3596.602294921875, 0.007216165773570538),
 (2246.435546875, 0.0027754181064665318),
 (2410.891357421875, 0.00043985299998894334),
 (2315.976318359375, 0.0004171373730059713),
 (2318.715087890625, 0.0012966420035809278),
 (2369.432861328125, 0.0008333539590239525)]